### Initial Analysis

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime


train_data = pd.read_csv("../data/landing/train.csv")
test_data = pd.read_csv("../data/landing/test.csv")

In [2]:
# Data Shape
[train_data.shape, test_data.shape]

[(10966, 24), (7312, 23)]

In [3]:
# Summary of train data
train_data.describe()

,TransactionNumber,UserID,Age,NumDependents,Latitude,Longitude,UserTenure,IsFraud
count,10966.000000,10966.000000,10966.000000,10966.000000,10923.000000,10923.000000,10966.000000,10966.000000
mean,9117.500091,2483.885282,2751.845887,1.995714,-30.363255,141.254786,60.786157,0.364308
std,5293.669640,1429.402615,9730.988917,1.408035,6.962819,11.268395,34.254477,0.481257
min,1.000000,1.000000,-68.000000,0.000000,-41.640079,-112.026050,1.000000,0.000000
25%,4516.250000,1253.000000,26.000000,1.000000,-37.020100,142.702789,31.000000,0.000000
50%,9120.500000,2471.000000,34.000000,2.000000,-31.840233,144.964600,61.000000,0.000000
75%,13710.750000,3727.000000,43.000000,3.000000,-25.042261,145.612793,90.000000,1.000000
max,18277.000000,5000.000000,67000.000000,4.000000,57.851580,149.012375,119.000000,1.000000


In [4]:
# Train data column types
train_data.dtypes

TransactionNumber        int64
UserID                   int64
Age                      int64
Gender                  object
Occupation              object
EducationLevel          object
MaritalStatus           object
NumDependents            int64
Income                  object
Expenditure             object
GiftsTransaction        object
TransactionDate         object
TransactionTime         object
TransactionAmount       object
MerchantID              object
TransactionType         object
TransactionLocation     object
DeviceType              object
Latitude               float64
Longitude              float64
EmailDomain             object
Terrorism                 bool
UserTenure               int64
IsFraud                  int64
dtype: object

### Preprocessing

In [5]:
# Cleaning the Age column
train_data['Age'] = train_data['Age'].apply(lambda x: x * -1 if x < 0 else (x / 1000 if x > 10000 else x))
test_data['Age'] = test_data['Age'].apply(lambda x: x * -1 if x < 0 else (x / 1000 if x > 10000 else x))

In [6]:
# Standardized Names of Locations
typical_locations = ['Adelaide', 'Canberra', 'Melbourne', 
                     'Sydney', 'Hobart', 'Darwin', 'Brisbane','Perth']

# All derivitions of the standardized location names
location_mapping = {
    'Adelaide City': 'Adelaide', 'Adl': 'Adelaide', 'adl': 'Adelaide',
    'BNE': 'Brisbane', 'Bne': 'Brisbane', 'brisbane': 'Brisbane',
    'CBR': 'Canberra', 'Cbr': 'Canberra', 'canberra': 'Canberra',
    'DRW': 'Darwin', 'Drw': 'Darwin', 'darwin': 'Darwin',
    'HBT': 'Hobart', 'Hbt': 'Hobart', 'hobart': 'Hobart',
    'MLB': 'Melbourne', 'Mel': 'Melbourne', 'Melb': 'Melbourne', 'Melburn': 'Melbourne',
    'melbourne': 'Melbourne',
    'PTH': 'Perth', 'Pth': 'Perth', 'perth': 'Perth',
    'SYD': 'Sydney', 'Syd': 'Sydney', 'sydney': 'Sydney'
}

# Create a new column TypicalLocationFlag
# This determines if a transaction location has the standardized name of the location
train_data['TypicalLocationFlag'] = train_data['TransactionLocation'].isin(typical_locations).astype(int)
test_data['TypicalLocationFlag'] = test_data['TransactionLocation'].isin(typical_locations).astype(int)

# Replaces all non-standard names with the standardized names
train_data['TransactionLocation'] = train_data['TransactionLocation'].replace(location_mapping)
test_data['TransactionLocation'] = test_data['TransactionLocation'].replace(location_mapping)

In [7]:
# This function requires the dataframe {df}, and a column name in the dataframe {column}
def changing_currency(df, column):
    """
    This function changes all different currency into AUD.
    """
    # Currency symbols found within the training data
    currency_symbols = {"AUD":["AU$", "AUD"], "GBP": ["GBP", "£"], "AED": ["AED"]}
    reverse_mapping = {symbol: currency for currency, symbols in currency_symbols.items() for symbol in symbols}

    numbers = []
    currencies = []

    # Iterates through all items
    for item in df[column]:
        # Find all amounts and currency symbols using re
        match = re.search(r'([\d,.]+)\s*(\D*)|(\D*)\s*([\d,.]+)', item)

        if match:
            number = match.group(1) or match.group(4)
            currency = match.group(2) or match.group(3)
            numbers.append(float(number))
            currencies.append(str(currency))

    # Creating a DataFrame with the amount and currency symbol
    new_df = pd.DataFrame({
        f'{column}_Amount': numbers,
        f'{column}_Currency': currencies,
    })

    # Remove all spaces within the currency symbols and changing to a standardized symbol
    new_df[f'{column}_Currency'] = new_df[f'{column}_Currency'].str.replace(' ', '', regex=False)
    new_df[f'{column}_Currency'] = new_df[f'{column}_Currency'].map(reverse_mapping)

    # Converting all currency to AUD
    GBP_to_AUD = 1.96
    AED_to_AUD = 0.41
    new_df.loc[new_df[f'{column}_Currency'] == 'GBP', f'{column}_Amount'] *= GBP_to_AUD
    new_df.loc[new_df[f'{column}_Currency'] == 'AED', f'{column}_Amount'] *= AED_to_AUD
    new_df = new_df.drop(columns=[f'{column}_Currency'])
    return new_df

In [8]:
# This function requires a dataframe with currency data columns
def changing_currency_columns(df):
    """
    This function changes all columns with currency data into AUD
    """
    # All columns with currency data
    currency_rows = ["Income", "Expenditure", "GiftsTransaction", "TransactionAmount"]

    for currency_row in currency_rows:
        changed_df = changing_currency(df, currency_row)
        df = pd.concat([df, changed_df], axis=1)

    # Remove all previous data with mix of currencies
    df = df.drop(columns=currency_rows)

    return df

In [9]:
# This function requires a string with time data
def standardize_time(time_str):
    """
    Many variations of Transaction time was found
    Therefore this function standardizes all time format to HH:MM:SS
    """
    try:
        # Trying 'hh/mm/ss' format (24-hour)
        time_obj = datetime.strptime(time_str, '%H/%M/%S')
        return time_obj.strftime('%H:%M:%S')
    except ValueError:
        try:
            # Trying 'hh:mm:ss AM/PM' format (12-hour with AM/PM)
            time_obj = datetime.strptime(time_str, '%I:%M:%S %p')
            return time_obj.strftime('%H:%M:%S')
        except ValueError:
            try:
                # Trying 'hh:mm:ss' format (24-hour)
                time_obj = datetime.strptime(time_str, '%H:%M:%S')
                return time_obj.strftime('%H:%M:%S')
            except ValueError:
                return None

In [10]:
# This function requires a dataframe with TransactionTime and TransactionDate
def normalize_date_time(df):
    """
    All timestamp data are separated into individual columns
    """
    # Time
    df['TransactionTime'] = df['TransactionTime'].apply(standardize_time)
    df['TransactionTime'] = pd.to_datetime(df['TransactionTime'], format='%H:%M:%S')
    df['Hour'] = df['TransactionTime'].dt.hour
    df['Minute'] = df['TransactionTime'].dt.minute

    # Dates
    df['TransactionDate'] = pd.to_datetime(df['TransactionDate'],format='%Y-%m-%d') 
    df['Year'] = df['TransactionDate'].dt.year
    df['Month'] = df['TransactionDate'].dt.month
    df['Day'] = df['TransactionDate'].dt.day

    # Removing obsolete columns
    df = df.drop(columns=["TransactionTime", "TransactionDate"])

    return df

In [11]:
# Cleaning the currency Columns
train_data = changing_currency_columns(train_data)
test_data = changing_currency_columns(test_data)

train_data = normalize_date_time(train_data)
test_data = normalize_date_time(test_data)

### Saving to /data/raw

In [12]:
import os

# Creating folder if folder does not exist
folder_path = '../data/raw'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Saving to /data/raw
train_file_path = os.path.join(folder_path, 'train_data_raw.csv')
test_file_path = os.path.join(folder_path, 'test_data_raw.csv')

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)